In [2]:
source ../config.sh

echo ${FD_BASE}
echo ${FD_WRK}
echo ${FD_LOG}
echo ${FD_ANN}

/work/kk319
/work/kk319/out/CombEffect_STARR
/work/kk319/out/CombEffect_STARR/log
/work/kk319/annotation


In [3]:
source ../config.sh
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
for SLURM_ARRAY_TASK_ID in {0..15}; do
    FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
    SAMPLE=$(basename ${FD_BED})
    
    echo ${FD_BED}
    echo ${SAMPLE}
done

/work/kk319/out/CombEffect_STARR/data/Input/
Input
/work/kk319/out/CombEffect_STARR/data/Input1/
Input1
/work/kk319/out/CombEffect_STARR/data/Input2/
Input2
/work/kk319/out/CombEffect_STARR/data/Input3/
Input3
/work/kk319/out/CombEffect_STARR/data/Input4/
Input4
/work/kk319/out/CombEffect_STARR/data/Input5/
Input5
/work/kk319/out/CombEffect_STARR/data/TFX2_DMSO/
TFX2_DMSO
/work/kk319/out/CombEffect_STARR/data/TFX2_Dex/
TFX2_Dex
/work/kk319/out/CombEffect_STARR/data/TFX3_DMSO/
TFX3_DMSO
/work/kk319/out/CombEffect_STARR/data/TFX3_Dex/
TFX3_Dex
/work/kk319/out/CombEffect_STARR/data/TFX4_DMSO/
TFX4_DMSO
/work/kk319/out/CombEffect_STARR/data/TFX4_Dex/
TFX4_Dex
/work/kk319/out/CombEffect_STARR/data/TFX5_DMSO/
TFX5_DMSO
/work/kk319/out/CombEffect_STARR/data/TFX5_Dex/
TFX5_Dex
/work/kk319/out/CombEffect_STARR/data/TFX_DMSO/
TFX_DMSO
/work/kk319/out/CombEffect_STARR/data/TFX_Dex/
TFX_Dex


In [16]:
### set environment on DCC
module load Bedtools
source ../config.sh

### run script using sbatch
#sbatch -pnew,all \
sbatch -p scavenger \
    --array=0-15 \
    --mem 8G \
    -o ${FD_LOG}/bootstrap_per1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories and global parameters
source ../config.sh
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
#TARGET=chr17
TARGET=target_PER1
SEED=123

### set input
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=${TARGET}.bed
N=$(cat ${FD_BED}/${FN_BED} | wc -l)

### set output
FD_OUT=${FD_WRK}/bootstrap/$(basename $FD_BED)
mkdir -p ${FD_OUT}

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Input  file:       " ${FD_BED}/${FN_BED}
echo "Total #Lines:      " ${N}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
head ${FD_BED}/${FN_BED}

### init: seed function
get_seeded_random()
{
  seed="$1";
  openssl enc -aes-256-ctr -pass pass:"$seed" -nosalt </dev/zero 2>/dev/null;
}

### loop: bootstrapping lines in input bed files
for i in {1..10}; do
    ### set output & print message
    FN_OUT=${TARGET}_B${i}.bed
    seed=$(( $SEED + $i ))
    echo
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo "Parameters:  SEED=${seed}" 
    
    ### random shuffle with replacement and sort
    shuf -rn ${N} ${FD_BED}/${FN_BED} --random-source=<(get_seeded_random $seed) > ${FD_OUT}/tmp.bed
    bedtools sort -i ${FD_OUT}/tmp.bed > ${FD_OUT}/${FN_OUT}
    
    ### print end message
    echo
    echo "Show the first few lines of the output file"
    head -n 3 ${FD_OUT}/${FN_OUT}
done

echo
echo "Done"

EOF

Submitted batch job 13988583


In [17]:
cat ${FD_LOG}/bootstrap_per1.3.txt

Slurm Array Index:  3
Input  file:        /work/kk319/out/CombEffect_STARR/data/Input3//target_PER1.bed
Total #Lines:       264

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input3//target_PER1.bed
chr17	8148017	8148918
chr17	8148022	8149011
chr17	8148153	8149179
chr17	8148189	8149070
chr17	8148194	8149156
chr17	8148316	8149209
chr17	8148316	8149211
chr17	8148551	8149463
chr17	8148573	8149682
chr17	8148643	8149665

Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input3/target_PER1_B1.bed
Parameters:  SEED=124

Show the first few lines of the output file
chr17	8148017	8148918
chr17	8148022	8149011
chr17	8148022	8149011

Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input3/target_PER1_B2.bed
Parameters:  SEED=125

Show the first few lines of the output file
chr17	8148017	8148918
chr17	8148017	8148918
chr17	8148022	8149011

Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input3/target_PER1_B3.bed
Parameters:  SEED=126

Show 

## Get the coverage

**Test loop**

In [23]:
source ../config.sh
FD_BEDS=($(ls -d ${FD_WRK}/bootstrap/*/))
TARGET=target_PER1

for SLURM_ARRAY_TASK_ID in {0..15}; do
    FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
    SAMPLE=$(basename ${FD_BED})
    
    echo ${FD_BED}
    echo ${SAMPLE}
    for FP_BED in $(ls /work/kk319/out/CombEffect_STARR/bootstrap/Input/${TARGET}_B*.bed); do
        echo ${FP_BED}
    done
done

/work/kk319/out/CombEffect_STARR/bootstrap/Input/
Input
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B1.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B10.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B2.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B3.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B4.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B5.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B6.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B7.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B8.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B9.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input1/
Input1
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B1.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B10.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B2.bed


In [22]:
TARGET=target_PER1
for FP_BED in $(ls /work/kk319/out/CombEffect_STARR/bootstrap/Input/${TARGET}_B*.bed); do
    echo ${FP_BED}
done

/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B1.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B10.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B2.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B3.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B4.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B5.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B6.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B7.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B8.bed
/work/kk319/out/CombEffect_STARR/bootstrap/Input/target_PER1_B9.bed


In [29]:
### set environment on DCC
module load Bedtools
source ../config.sh

### run script using sbatch
#sbatch -pnew,all \
sbatch -p scavenger \
    --array=0-15 \
    --mem 8G \
    -o ${FD_LOG}/coverage_per1_bootstrap.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories and global parameters
source ../config.sh
FD_BEDS=($(ls -d ${FD_WRK}/bootstrap/*/))
CHROM=chr17
TARGET=target_PER1
SEED=123

### set input and output
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FD_GEN=${FD_ANN}/genome
FN_GEN=hg38.chrom.sizes.chr17

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop: bootstrapping lines in input bed files
for i in {1..10}; do
    ### set output & print message
    FN_BED=${TARGET}_B${i}.bed
    FN_OUT=${TARGET}_B${i}_cov.txt
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_BED}/${FN_OUT}
    
    ### print end message
    echo
    echo "Show the first few lines of the input file"
    head -n 3 ${FD_BED}/${FN_BED}
    
    ### get coverage
    bedtools genomecov -i ${FD_BED}/${FN_BED} -g ${FD_GEN}/${FN_GEN} -d > ${FD_BED}/${FN_OUT}
    
    ### print end message
    echo
    echo "Show the first few lines of the output file"
    head -n 3 ${FD_BED}/${FN_OUT}
done

echo
echo "Done"

EOF

Submitted batch job 13989003


In [32]:
cat ${FD_LOG}/coverage_per1_bootstrap.1.txt

Slurm Array Index:  1
Input  file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B1.bed
Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B1_cov.txt

Show the first few lines of the input file
chr17	8148122	8149107
chr17	8148122	8149107
chr17	8148188	8149154

Show the first few lines of the output file
chr17	1	0
chr17	2	0
chr17	3	0
Input  file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B2.bed
Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B2_cov.txt

Show the first few lines of the input file
chr17	8148117	8149012
chr17	8148117	8149012
chr17	8148190	8149151

Show the first few lines of the output file
chr17	1	0
chr17	2	0
chr17	3	0
Input  file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B3.bed
Output file:  /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B3_cov.txt

Show the first few lines of the input file
chr17	8148117	8149012
chr17	8148178	8149194
chr1

In [2]:
cat /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B10_cov.txt |\
    head -n 8148000 | tail

chr17	8147991	0
chr17	8147992	0
chr17	8147993	0
chr17	8147994	0
chr17	8147995	0
chr17	8147996	0
chr17	8147997	0
chr17	8147998	0
chr17	8147999	0
chr17	8148000	0
cat: write error: Broken pipe


In [3]:
cat /work/kk319/out/CombEffect_STARR/bootstrap/Input1//target_PER1_B10_cov.txt |\
    head -n 8148010 | tail

chr17	8148001	0
chr17	8148002	0
chr17	8148003	0
chr17	8148004	0
chr17	8148005	0
chr17	8148006	0
chr17	8148007	0
chr17	8148008	0
chr17	8148009	0
chr17	8148010	0
cat: write error: Broken pipe
